Aprendizaje Automático - Práctica 2
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 1. K-Means
---
Programamos el algoritmo K-means y comparamos nuestra implementación con la implementación de sklearn

In [24]:
# Importamos
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale, OneHotEncoder, OrdinalEncoder
# import matplotlib.pylot as plt

# Cargamos los datos
data = pd.read_csv('data/stars_data.csv')

# Definimos nuestro conjunto de datos
X = data.values

# Copiamos el conjunto de datos original para preservarlo
X_onehot = X.copy()
X_ordinal = X.copy()

# Codificamos valores de tipo str (color y clase espectral)
# One-Hot Encoder
encoder_onehot = OneHotEncoder(sparse_output=False)
X_onehot_encoded = encoder_onehot.fit_transform(X_onehot[:, -2:])

# Eliminamos filas tipo str - One Hot
X_onehot = X_onehot[:, :-2]

# Concatenamos las nuevas filas codificadas - One Hot
X_onehot = np.concatenate((X_onehot, X_onehot_encoded), axis=1)

# Ordinal Encoder
encoder_ordinal = OrdinalEncoder()
X_ordinal_encoded = encoder_ordinal.fit_transform(X_onehot[:, -2:])

# Eliminamos filas tipo str - Ordinal
X_ordinal = X_ordinal[:, :-2]

# Concatenamos las nuevas filas codificadas - Ordinal
X_ordinal = np.concatenate((X_ordinal, X_ordinal_encoded), axis=1)

# Escalamos los datos
X_onehot_scaled = scale(X_onehot)
X_ordinal_scaled = scale(X_ordinal)

from sklearn.cluster import KMeans

# Definimos el número de clusters
n_clusters = 4

# KMeans con one hot encoded data
kmeans_onehot = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_onehot.fit(X_onehot_scaled)

# KMeans con ordinal encoded data
kmeans_ordinal = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_ordinal.fit(X_ordinal_scaled)

# Obtenemos los labels de los clusters
labels_onehot = kmeans_onehot.labels_
labels_ordinal = kmeans_ordinal.labels_

# Imprimimos los centroides
print("Cluster centers for one-hot encoded data:")
print(kmeans_onehot.cluster_centers_)
print("\nCluster centers for ordinal encoded data:")
print(kmeans_ordinal.cluster_centers_)




Cluster centers for one-hot encoded data:
[[-0.41884518 -0.52136177 -0.45337807  0.70720499 -0.33823072  0.13900961
   0.08679261  0.04312308 -0.3485618  -0.09166985  0.04312308  0.04312308
   0.30345287  0.03301503  0.04312308  0.06111323  0.04312308  0.07500586
   0.07500586  0.12379689  0.06111323 -0.06172869 -0.04587143  0.18406905
  -0.06468462  0.01779202  0.31198982 -0.4472136 ]
 [ 0.1107731   0.44081553  1.19504385 -1.0745122  -0.50789337 -0.20851441
  -0.13018891 -0.06468462  1.20060175  0.3157517  -0.06468462 -0.06468462
  -0.08166316  0.0467713  -0.06468462 -0.09166985 -0.06468462 -0.11250879
  -0.11250879 -0.18569534 -0.09166985  0.39266305  0.40696195 -0.27610357
   0.22280259  0.07709874 -0.11049639 -0.4472136 ]
 [ 1.29361805  1.22076325  0.01795413 -1.04318715  1.81265393 -0.20851441
  -0.13018891 -0.06468462 -0.3485618  -0.09166985 -0.06468462 -0.06468462
  -0.93541435 -0.17332893 -0.06468462 -0.09166985 -0.06468462 -0.11250879
  -0.11250879 -0.18569534 -0.09166985 -0.2